In [1]:
using Random, Distributions, Statistics
using Plots

In [ ]:
# DA algorithm with "keep"-idea
# acceptor preference must be a dictionary of tupples of the proposer and its preference order, i.e. [("A", 1), ("B", 2)]

function DA_keep(props, accs, prop_prefs, acc_prefs, acc_cap)
    prop_prefs_copy = deepcopy(prop_prefs) # Dictionary for each acceptor
    free_props = Set(props) 
    acc_matches = Dict(a => [] for a in accs) # keep -> formal matching
    prop_matches = Dict(a => [] for a in props)
    acc_matches["outside"] = [] # outside option
    prop_matches["outside"] = []
    acc_applylist = deepcopy(acc_matches) # Dictionary for each acceptor

    algo_lim = 5000 # limit in case of too long iteration
    count = 1

    while !isempty(free_props) && count < algo_lim # continue until everybody gets matched
        for prop in free_props
            apply = first(prop_prefs_copy[prop]) # most prefarable one at this time
            for j in acc_prefs[apply]
                if j[1] == prop # check the individual rationality
                    push!(acc_applylist[apply], j)
                end
            end
            prop_prefs_copy[prop] = setdiff(prop_prefs_copy[prop], [apply]) # delete the acceptor from the proposer's preference list(copy)
        end
        # acc_applylist is filled 
        for acc in accs
            if acc_applylist[acc] !== acc_matches[acc] # only conducts the algorithm if the kept proposers profile is different from the previous 
                sort!(acc_applylist[acc], by = x -> x[2])

                k = min(acc_cap[acc], length(acc_applylist[acc])) # not to exceed the capacity, also technically not to exceed the current matches
                acc_matches[acc] = acc_applylist[acc][1:k] #"keep"

                for kept in acc_matches[acc] # delete proposers from the free propser set
                    delete!(free_props, kept[1])
                end

                for rejected in acc_applylist[acc][k+1:end] # 
                    if isempty(prop_prefs_copy[rejected[1]])
                        push!(prop_matches["outside"], rejected[1])
                        delete!(free_props, rejected[1])
                    else
                        push!(free_props, rejected[1])
                    end
                end
            end
            acc_applylist[acc] = deepcopy(acc_matches[acc])
        end
        count += 1
        if count == algo_lim
            print("reached algo_lim")
        end
    end

    for acc in accs
        acc_matches[acc] = [k[1] for k in acc_matches[acc]]
        for i in acc_matches[acc]
            push!(prop_matches[i], acc)
        end
    end
    return prop_matches, acc_matches
end

DA_keep (generic function with 1 method)

In [3]:
function DA_onebyone(props, accs, prop_prefs, acc_prefs, acc_cap)
    prop_prefs_copy = deepcopy(prop_prefs)
    free_props = Set(props)
    acc_matches = Dict(a => [] for a in accs)
    prop_matches = Dict(a => [] for a in props)
    acc_matches["outside"] = []
    prop_matches["outside"] = []
    algo_lim = 5000
    count = 1

    while !isempty(free_props) && count < algo_lim
        for prop in free_props
            apply = first(prop_prefs_copy[prop])
            if in(prop, acc_prefs[apply])
                if length(acc_matches[apply]) < acc_cap[apply]
                    push!(acc_matches[apply], prop)
                    push!(prop_matches[prop], apply)
                    delete!(free_props, prop)
                else
                    worst_match = 1
                    for current_match in acc_matches[apply]
                        order = findfirst(==(current_match), acc_prefs[apply])
                        if order > worst_match
                            worst_match = order
                        end
                    end
                    if findfirst(==(prop), acc_prefs[apply]) < worst_match
                        rejected = acc_prefs[apply][worst_match]
                        #if acceptor prefers the new prop
                        filter!(e -> e ≠ rejected, acc_matches[apply])
                        push!(acc_matches[apply], prop)
                        push!(prop_matches[prop], apply)
                        delete!(free_props, prop)
                        prop_matches[rejected] = []
                        if isempty(prop_prefs_copy[rejected])
                            push!(prop_matches["outside"], prop)
                        else
                            push!(free_props, rejected)
                        end
                    end
                end
            end
            prop_prefs_copy[prop] = setdiff(prop_prefs_copy[prop], [apply])
            if isempty(prop_prefs_copy[prop]) && isempty(prop_matches[prop])
                push!(prop_matches["outside"], prop)
                delete!(free_props, prop)
            end
        end
        count += 1
        if count == algo_lim
            print("reached algo_lim")
        end
    end
    return prop_matches, acc_matches
end

DA_onebyone (generic function with 1 method)

In [3]:
function marketgen(n,m,app_min, app_max, acc_min, acc_max, uni_pref_max, ud)
    stu = ["$i" for i in 1:n] #学生
    uni = ["$j" for j in 1001:1000+m] #大学
    stu_pref = Dict(s => [] for s in stu) #学生の選好
    uni_pref = Dict(u => [] for u in uni) #大学の選好
    uni_cap = Dict(u => rand(acc_min:acc_max) for u in uni) #大学の受け入れ可能数
    stu_cap = Dict(s => 1 for s in stu) #学生の受け入れ可能数 = 1


    #ランダムに選好を生成する（ソート）
    for s in stu
        app_num = rand(app_min:app_max) #応募大学数
        rand_pref = [[rand(ud) j] for j in uni] 
        sort!(rand_pref, by= x->x[1])
        stu_pref[s] = [rand_pref[k][2] for k in 1:app_num]
    end

    for u in uni
        rand_pref = [[rand(ud) j] for j in stu] 
        pref_max = uni_pref_max #大学の選好リスト長さ
        sort!(rand_pref, by= x->x[1])
        uni_pref[u] = [rand_pref[k][2] for k in 1:pref_max]
    end
    return stu, uni, stu_pref, uni_pref, stu_cap, uni_cap
end

marketgen (generic function with 1 method)

In [4]:
function marketgen_keep(n,m,app_min, app_max, acc_min, acc_max, uni_pref_max, ud)
    stu = ["$i" for i in 1:n] #学生
    uni = ["$j" for j in 1001:1000+m] #大学
    stu_pref = Dict(s => [] for s in stu) #学生の選好
    uni_pref = Dict(u => [] for u in uni) #大学の選好
    uni_cap = Dict(u => rand(acc_min:acc_max) for u in uni) #大学の受け入れ可能数
    stu_cap = Dict(s => 1 for s in stu) #学生の受け入れ可能数 = 1


    #ランダムに選好を生成する（ソート）
    for s in stu
        app_num = rand(app_min:app_max) #応募大学数
        rand_pref = [[rand(ud) j] for j in uni] 
        sort!(rand_pref, by= x->x[1])
        stu_pref[s] = [rand_pref[k][2] for k in 1:app_num]
    end

    for u in uni
        rand_pref = [[rand(ud) j] for j in stu] 
        pref_max = uni_pref_max #大学の選好リスト長さ
        sort!(rand_pref, by= x->x[1])
        uni_pref[u] = [(rand_pref[k][2], k) for k in 1:pref_max]
    end
    return stu, uni, stu_pref, uni_pref, stu_cap, uni_cap
end

marketgen_keep (generic function with 1 method)

In [6]:
#多対一DAを回してC(n)を計算
function Cn(stu, uni, stu_pref, uni_pref, stu_cap, uni_cap)
    #学生応募制
    matches_so = DA(stu, uni, stu_pref, uni_pref, stu_cap, uni_cap)

    #大学応募制
    matches_uo = DA(uni, stu, uni_pref, stu_pref, uni_cap, stu_cap)
    C = 0
    for s in stu
        if !(matches_so[1][s] == matches_uo[2][s])
            C += 1
        end
    end
    return C/size(stu, 1)
end
    

Cn (generic function with 1 method)

In [7]:
# Student-applying Deferred Acceptance Algorithm in Julia

function DA(props, accs, prop_prefs, acc_prefs, prop_cap, acc_cap)
    prop_prefs_copy = copy(prop_prefs)
    free_props = Set(props)
    acc_matches = Dict(a => [] for a in accs)
    prop_matches = Dict(a => [] for a in props)
    acc_matches["outside"] = []
    prop_matches["outside"] = []
    algo_lim = 10000
    count = 1

    while !isempty(free_props) && count < algo_lim
        for prop in free_props
            apply = first(prop_prefs_copy[prop])
            if in(prop, acc_prefs[apply])
                if length(acc_matches[apply]) < acc_cap[apply]
                    push!(acc_matches[apply], prop)
                    push!(prop_matches[prop], apply)
                    delete!(free_props, prop)
                else
                    worst_match = 1
                    for current_match in acc_matches[apply]
                        order = findfirst(==(current_match), acc_prefs[apply])
                        if order > worst_match
                            worst_match = order
                        end
                    end
                    if findfirst(==(prop), acc_prefs[apply]) < worst_match
                        rejected = acc_prefs[apply][worst_match]
                        #if acceptor prefers the new prop
                        filter!(e -> e ≠ rejected, acc_matches[apply])
                        push!(acc_matches[apply], prop)
                        push!(prop_matches[prop], apply)
                        delete!(free_props, prop)
                        prop_matches[rejected] = []
                        if isempty(prop_prefs_copy[rejected])
                            push!(prop_matches["outside"], prop)
                        else
                            push!(free_props, rejected)
                        end
                    end
                end
            end
            prop_prefs_copy[prop] = setdiff(prop_prefs_copy[prop], [apply])
            if isempty(prop_prefs_copy[prop]) && isempty(prop_matches[prop])
                push!(prop_matches["outside"], prop)
                delete!(free_props, prop)
            end
        end
        count += 1
    end
    return prop_matches, acc_matches
end

DA (generic function with 2 methods)

In [ ]:
#C(n) をプロットする
n_max = 300 #市場規模最大値
knum_max = 3 #選好リストの数の種類最大値
S = 5 #各値での試行回数
x = zeros(n_max, knum_max)

app_min = 4 #学生が応募する大学数下限
app_max = 15 #学生が応募する大学数上限
acc_min = 1 #大学の受け入れ可能学生数下限
acc_max = 1 #大学の受け入れ可能学生数上限
ud = Uniform(0,1)
Random.seed!(1234)

for k in 1:knum_max
    k_5 = 5 * k + 5
    for n in k_5:n_max
        CC = 0
        for j in 1:S
            market = marketgen(n, n, min(k_5,15), min(k_5,15), acc_min, acc_max, n, Uniform(0,1))
            CC += Cn(market[1], market[2], market[3], market[4], market[5],market[6])
        end
        x[n, k] = CC /S
    end
    print(k_5, " end \n")
end

10 end 


In [ ]:
(n,k)= size(x)
y = copy(x)
for j in 1:k
    k_5 = 5*j+5
    
    for i in k_5+1:n-1
        y[i,j] = mean(x[i-1:i+1,j])
    end
end

In [29]:
ENV["GKS_ENCODING"] = "utf8"
gr(fontfamily="IPAMincho")
plot([10:n_max], y[10:n_max, 1], label = "k=10, 1to1", title = "戦略的操作が可能な参加者割合と \n 市場規模 n・選好リストの長さ k・ \n 大学受け入れ可能数の関係", xlab = "n", ylab = "C(n)/n", xscale=:log10)
plot!([10:n_max],y_2[10:n_max, 1], label = "k=10, 1to2")
plot!([15:n_max],y[15:n_max,2], label = "k=15, 1to1")
plot!([15:n_max],y_2[15:n_max,2], label = "k=15, 1to2")
plot!([20:n_max],y[20:n_max,3], label = "k=20, 1to1")
plot!([20:n_max],y_2[20:n_max,3], label = "k=20, 1to2")
#plot!([25:n_max],y[25:n_max,4], label = "k=25")
#plot!([30:n_max],y[30:n_max,5], label = "k=30")
savefig("matching_sim_3.svg")

"g:\\マイドライブ\\1. Univ_docs\\経研勉強会\\マッチング\\code\\matching_sim_3.svg"